In [1]:
import numpy as np
import torch
from torch.utils import data

简洁实现就是直接调包，不用自己写一些函数实现

In [2]:
def synthetic_data(w,b,num_examples):
    X=torch.normal(0,1,(num_examples,len(w)))
    y=torch.matmul(X,w)+b
    y+=torch.normal(0,0.01,y.shape)
    return X,y.reshape((-1,1))

In [3]:
true_w=torch.tensor([2,-3.4])
true_b=4.2

features, labels=synthetic_data(true_w,true_b,1000)

## 调用框架中现有的API来读取数据

In [4]:
def load_array(data_arrays,batch_size,is_train=True):
    """
    构建一个pytorch数据生成器
    """
    dataset=data.TensorDataset(*data_arrays)
    print(data_arrays[0][0].size(0),data_arrays[0][23].size(0))
    # 第二个是任意一个序号样本 对features和labels分别进行这样的验证
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

batch_size=10
data_iter=load_array((features,labels),batch_size)

next(iter(data_iter))
# 调用一次next，就会产生一批数据，
# 和yield功能差不多，yield内部也实现了next函数

2 2


[tensor([[-0.6819, -0.2478],
         [ 0.0876, -0.3977],
         [ 0.5512,  1.0308],
         [-0.7333,  1.5255],
         [ 0.2307,  0.7130],
         [-0.6225, -0.1732],
         [-1.0890, -0.0059],
         [-0.5752,  0.3171],
         [ 0.3993, -0.1051],
         [-1.5982, -0.6899]]),
 tensor([[ 3.6711],
         [ 5.7236],
         [ 1.7837],
         [-2.4466],
         [ 2.2440],
         [ 3.5534],
         [ 2.0515],
         [ 1.9648],
         [ 5.3541],
         [ 3.3623]])]

In [5]:
all(features[0].size(0) == tensor.size(0) for tensor in features)
print(features[0].size(0))

# 整个输入特征的第一维，是每个样本的特征数量，判断每个样本的特征数量是否等于 整个输入特征的第一维
# 以第一维 的序号作为这个样本的检索标准

2


    *tensors (Tensor): tensors that have the same size of the first dimension.

看了一下对应的函数：https://pytorch.org/docs/stable/_modules/torch/utils/data/dataset.html#TensorDataset
```python
[docs]class TensorDataset(Dataset[Tuple[Tensor, ...]]):
    r"""Dataset wrapping tensors.

    Each sample will be retrieved by indexing tensors along the first dimension.

    Args:
        *tensors (Tensor): tensors that have the same size of the first dimension.
    """
    tensors: Tuple[Tensor, ...]

    def __init__(self, *tensors: Tensor) -> None:
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors), "Size mismatch between tensors"
        self.tensors = tensors

    def __getitem__(self, index):
        return tuple(tensor[index] for tensor in self.tensors)

    def __len__(self):
        return self.tensors[0].size(0)
```

最重要的一句话：每个样本将会沿着第一维度的索引被检索。

## 模型定义

In [6]:
# 使用框架预定义好的层
from torch import nn

net=nn.Sequential(nn.Linear(2,1))

# 初始化模型参数
# print(net[0].weight,net[0].weight.data)
# print(net[0].bias,net[0].bias.data)

net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

# pytoch中下划线表示 in-place操作

# print("\n")
# print(net[0].weight,net[0].weight.data)
# print(net[0].bias,net[0].bias.data)

# 权重初始化前后值不同标明，在构建Sequential时，w和b就被赋值了

# MSELoss，平方L2范数，就是之前的均方差Mean Square Error loss 
loss=nn.MSELoss()

# 实例化SGD实例
trainer=torch.optim.SGD(net.parameters(),lr=0.03)
# 需要传入至少两个参数，一是要求梯度下降的超参，一个是学习率

num_epochs=3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l=loss(net(X),y)
        trainer.zero_grad()
        l.backward()
        print(l)
        trainer.step()
    l=loss(net(features),labels)
    print(f"epoch {epoch+1},loss {l:f}")

C:\software\anaconda\lib\site-packages\torch\autograd\__init__.py:132: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


tensor(11.7394, grad_fn=<MseLossBackward>)
tensor(21.1278, grad_fn=<MseLossBackward>)
tensor(25.6993, grad_fn=<MseLossBackward>)
tensor(20.5999, grad_fn=<MseLossBackward>)
tensor(14.2759, grad_fn=<MseLossBackward>)
tensor(20.4857, grad_fn=<MseLossBackward>)
tensor(25.5462, grad_fn=<MseLossBackward>)
tensor(6.3983, grad_fn=<MseLossBackward>)
tensor(18.1160, grad_fn=<MseLossBackward>)
tensor(15.3537, grad_fn=<MseLossBackward>)
tensor(14.3052, grad_fn=<MseLossBackward>)
tensor(5.8712, grad_fn=<MseLossBackward>)
tensor(6.2474, grad_fn=<MseLossBackward>)
tensor(5.4770, grad_fn=<MseLossBackward>)
tensor(13.1340, grad_fn=<MseLossBackward>)
tensor(6.7820, grad_fn=<MseLossBackward>)
tensor(5.4515, grad_fn=<MseLossBackward>)
tensor(11.0596, grad_fn=<MseLossBackward>)
tensor(2.1408, grad_fn=<MseLossBackward>)
tensor(4.2195, grad_fn=<MseLossBackward>)
tensor(3.8079, grad_fn=<MseLossBackward>)
tensor(3.4709, grad_fn=<MseLossBackward>)
tensor(2.7187, grad_fn=<MseLossBackward>)
tensor(2.2596, grad_fn

平均平方误差，均方误差MSE mean square error
$$cost(w)=\frac{1}{N}\sum_{n-1}^N(\hat{y}_n-y_n)^2$$

## MSEloss官方实现

https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html

可以看到在官方实现中，使用默认计算每个样本损失后，进行mean，之前在从头实现里，使用的是sum()

# 优化器.step()

    trainer=torch.optim.SGD(net.parameters(),lr=0.03)
    trainer.step()

https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.step.html#torch.optim.Optimizer.step

"""Performs a single optimization step (parameter update)."""

更新一次梯度

## linear层定义（官方文档）

    nn.Linear(in_features: int, out_features: int, bias: bool = True) -> None
    
输入特征维度：2
输出特征维度：1

线性层就是全连接了，代码：https://pytorch.org/docs/stable/_modules/torch/nn/modules/linear.html

主要这里的说明：

```python
"""Applies a linear transformation to the incoming data: :math:`y = xA^T + b`

    This module supports :ref:`TensorFloat32<tf32_on_ampere>`.

    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        bias: If set to ``False``, the layer will not learn an additive bias.
            Default: ``True``

    Shape:
        - Input: :math:`(N, *, H_{in})` where :math:`*` means any number of
          additional dimensions and :math:`H_{in} = \text{in\_features}`
        - Output: :math:`(N, *, H_{out})` where all but the last dimension
          are the same shape as the input and :math:`H_{out} = \text{out\_features}`.
```

就是应用线性变化，公式就是y=ax+b，输入就是特征维度（features），输出的最后一个维度和输入特征的最后一个维度（即。样本数量）一样。

## 参数初始化

---

关于使用某种分布初始化参数：

https://pytorch.org/docs/stable/nn.init.html

stack overflow：
+ [How to initialize weights in PyTorch?](https://stackoverflow.com/questions/49433936/how-to-initialize-weights-in-pytorch)


Single layer
To initialize the weights of a single layer, use a function from torch.nn.init. For instance:

```
conv1 = torch.nn.Conv2d(...)
torch.nn.init.xavier_uniform(conv1.weight)
```

Alternatively, you can modify the parameters by writing to conv1.weight.data (which is a torch.Tensor). Example:

```
conv1.weight.data.fill_(0.01)
# The same applies for biases:
conv1.bias.data.fill_(0.01)
```